In [111]:
import csv

In [112]:
import pandas as pd

In [113]:
import glob

In [114]:
import os

In [115]:
## Reads in a dataset of legislators from govtrack.com's GitHub.
legislator_info = pd.read_csv('summary.csv')

In [116]:
legislator_info.drop(columns=['pct_true', 'count_all'],inplace=True)
legislator_info.info()

###  Now we have a spreadsheet with all the names of legislators and some data about them 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   full_name   526 non-null    object 
 1   gender      526 non-null    object 
 2   party       526 non-null    object 
 3   type        526 non-null    object 
 4   birthday    526 non-null    object 
 5   state       526 non-null    object 
 6   district    427 non-null    float64
 7   generation  525 non-null    object 
dtypes: float64(1), object(7)
memory usage: 33.0+ KB


In [117]:
### Now I need to import my 200 tweets per legislator ###
file_list = sorted(glob.glob('./legislator_tweets/*.csv'))

In [118]:
### Now I need to loop through each file in file_list and read thru all rows to check for China ###

In [119]:
# loop through all the file names
summary_data = []
for file in file_list: 
  # open csv
  with open(file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # establish fresh values
    count_all = 0
    mentions_china = 0
    pct_true = 0.0

    # loop through all the rows in the csv
    for row in reader:

      # skip this row if there's no content
      if row['tweet'] == "":
        continue 

      # we have content, count this row
      count_all += 1

      #is China mentioned?
      if 'China' in row['tweet'] or 'Chinese' in row['tweet'] or 'china' in row['tweet'] or 'chinese' in row['tweet']:
        mentions_china += 1 
        
    short_file = os.path.basename(file)
    
    this_item = [short_file, count_all, mentions_china]
    #print(this_item)

    summary_data.append(this_item)



In [120]:
### Ok so the function above reads every row in each file, then spits out one row with name,
### .total tweets, and the number of tweets with China mentioned!

In [121]:
### Change that list of lists to a dataframe 
china_mentions = pd.DataFrame(summary_data, columns=['file', 'count_all', 'mentions_china'])

In [122]:
###  Need to take off the '.csv' there so I can merge it with my original data.  
china_mentions['file'] = china_mentions['file'].str.replace('.csv','')

In [123]:
### The merge !   
merged = china_mentions.merge(legislator_info, left_on='file', right_on='full_name')

In [124]:
###   The clean-up....
merged.drop(columns=['file'], inplace=True)
master = merged

In [132]:
### This is the data I will work with.
master.head()

,count_all,mentions_china,full_name,gender,party,type,birthday,state,district,generation
0,118,0,A. Donald McEachin,M,Democrat,rep,1961-10-10,VA,4.0,Boomers
1,133,5,A. Drew Ferguson IV,M,Republican,rep,1966-11-15,GA,3.0,Gen X
2,151,2,Abby Finkenauer,F,Democrat,rep,1988-12-27,IA,1.0,Millennial
3,153,0,Abigail Davis Spanberger,F,Democrat,rep,1979-08-07,VA,7.0,Gen X
4,195,4,Adam B. Schiff,M,Democrat,rep,1960-06-22,CA,28.0,Boomers


In [126]:
## Questions:
## Which congressperson mentions China the absolute most? Out of their total tweets, how many times did they mention it?
## Which party talks about China the most?
## Who talks about China more, the House or the Senate?
## Which Age groups talks about China the most? (Might not be significant)

In [134]:
## Question 1: ## Which congressperson mentions China the absolute most?
##    sub question: Out of their total tweets, how many times did they mention it?


In [156]:
master.sort_values('mentions_china', ascending=False).reset_index(drop=True)

,count_all,mentions_china,full_name,gender,party,type,birthday,state,district,generation
0,143,75,Tom Cotton,M,Republican,sen,1977-05-13,AR,NaN,Gen X
1,145,60,Marsha Blackburn,F,Republican,sen,1952-06-06,TN,NaN,Boomers
2,165,56,Mark E. Green,M,Republican,rep,1964-11-08,TN,7.0,Boomers
3,145,55,Jim Banks,M,Republican,rep,1979-07-16,IN,3.0,Gen X
4,139,47,Michael Waltz,M,Republican,rep,1974-01-31,FL,6.0,Gen X
...,...,...,...,...,...,...,...,...,...,...
521,179,0,Jared Huffman,M,Democrat,rep,1964-02-18,CA,2.0,Boomers
522,94,0,Jared F. Golden,M,Democrat,rep,1982-07-25,ME,2.0,Millennial
523,84,0,Janice D. Schakowsky,F,Democrat,rep,1944-05-26,IL,9.0,Boomers
524,53,0,Jamie Raskin,M,Democrat,rep,1962-12-13,MD,8.0,Boomers


In [144]:
## Answer: Tom Cotton mentions China the most. That's not too surprising, he was the one pushing that bioweapons theory in April.

In [200]:
## Which party talks about China the most?
party_mentions = master.groupby("party")['mentions_china'].sum()
party_mentions

party
Democrat        121
Independent       1
Republican     1230
Name: mentions_china, dtype: int64

In [203]:
## Who talks about China more, the House or the Senate?
type_mentions = master.groupby("type")['mentions_china'].sum()
type_mentions

type
rep    952
sen    400
Name: mentions_china, dtype: int64

In [208]:
## Members of the house mentioned China 2.4 times as much as members of the Senate.

In [207]:
## Which Age groups tweeted about China the most? (Might not be significant)
age_mentions = master.groupby("generation")['mentions_china'].sum()
age_mentions

generation
Boomers       633
Gen X         588
Millennial    100
Silent         31
Name: mentions_china, dtype: int64